In [1]:
!pip install transformers

     |████████████████████████████████| 1.4MB 13.1MB/s 
     |████████████████████████████████| 2.9MB 23.5MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2271e499078d4b84631c0d6246dc3346e306b09123925b202d96ffb55d49df7f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

2020-12-10 21:56:58.991641: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 629/629 [00:00<00:00, 1.04MB/s]
Downloading: 100% 268M/268M [00:03<00:00, 74.1MB/s]
Downloading: 100% 232k/232k [00:00<00:00, 15.2MB/s]
[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [ ]:
#  from transformers import BertForMaskedLM
#  import torch
#  model = BertForMaskedLM.from_pretrained("bert-base-uncased")

In [65]:
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

In [20]:
!pip install jsonlines
import jsonlines
from sklearn.model_selection import train_test_split
!pip install python-levenshtein
import Levenshtein as Lev
import torch
from torch.utils.data import Dataset as TorchDataset


     |████████████████████████████████| 51kB 8.5MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144797 sha256=e3b604cf448ac88ac965a483c5eda7df6e8d7ef3e1417081342af03df427c2f6
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [25]:
from typing import Sequence, Dict, Any, List
import json

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'trec')

     |████████████████████████████████| 163kB 21.9MB/s 
     |████████████████████████████████| 245kB 41.6MB/s 
     |████████████████████████████████| 17.7MB 218kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


Using custom data configuration default


Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48. Subsequent calls will reuse this data.


In [6]:
from transformers import BertTokenizerFast, BertForMaskedLM
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [7]:
from transformers import AutoTokenizer
dataset = load_dataset('trec', split='test')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

Using custom data configuration default
Reusing dataset trec (/root/.cache/huggingface/datasets/trec/default/1.1.0/1902c380fe66cc215f989888b1b35e8da7e79a3a97520f00dce753fd1f8f5c48)


In [52]:
dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'label-coarse', 'label-fine', 'text', 'token_type_ids'],
    num_rows: 500
})

In [53]:
dataset.column_names

['attention_mask',
 'input_ids',
 'label-coarse',
 'label-fine',
 'text',
 'token_type_ids']

In [54]:
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label-coarse'])
# dataset.set_format(type='torch')

In [55]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
next(iter(dataloader))

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  2129,  2521,  ...,     0,     0,     0],
         [  101,  2054,  2221,  ...,     0,     0,     0],
         [  101,  2040,  2001,  ...,     0,     0,     0],
         ...,
         [  101,  2054, 14130,  ...,     0,     0,     0],
         [  101,  2054,  2003,  ...,     0,     0,     0],
         [  101,  2073,  2024,  ...,     0,     0,     0]]),
 'label-coarse': tensor([4, 5, 3, 0, 4, 4, 3, 1, 0, 0, 5, 3, 4, 3, 4, 4, 1, 3, 0, 4, 3, 0, 5, 0,
         0, 3, 0, 5, 5, 5, 4, 5]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]

In [56]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [57]:
import numpy as np
import random 


def mask_tokens(batch, tokenizer):
    batch_masked, masked_inds = [], []
    for b in batch:
        inds_to_mask = np.random.choice(torch.arange(1, (b[b!=0].shape[0] -1)), size=1)
        masked_inds.append(inds_to_mask)
        b_new = b.clone()
        b_new[inds_to_mask] = tokenizer.mask_token_id
        batch_masked.append(b_new)
    return torch.stack(batch_masked), masked_inds

In [58]:
for batch in dataloader:
    with torch.no_grad():
        
        b_input_ids = batch['input_ids']
        b_input_mask = batch['attention_mask']
        b_token_type_ids = batch['token_type_ids']
        b_labels = batch['label-coarse']
        b_masked, masked_inds = mask_tokens(b_input_ids, tokenizer)
        logits = model(b_masked, attention_mask=b_input_mask)
        break

In [60]:
logits['logits'][0, :, :].shape

torch.Size([512, 30522])

In [61]:
logits[0][0].shape

torch.Size([512, 30522])

In [62]:
generated_batch = []
original_batch = []
generated_batch_sentence = []
original_batch_sentence = []
for i in range(32):
    generated = (torch.argmax(torch.softmax(logits[0][i], axis=1), axis=1) * batch['attention_mask'][i])
    original = b_input_ids[i]
    print('Original:')
    print(tokenizer.decode(original[(original!=0) & (original!=101) & (original!=102)]))
    print('Generated:')
    print(tokenizer.decode(generated[(generated != 0) & (generated!=1012)]))
    print('Masked word original:', tokenizer.decode(original[original!=0][masked_inds[i]]))
    print('Masked word prediction:', tokenizer.decode(generated[generated!=0][masked_inds[i]]))
    print('$'*40)
    original_batch_sentence.append(tokenizer.decode(original[(original!=0) & (original!=101) & (original!=102)]))
    generated_batch_sentence.append(tokenizer.decode(generated[(generated != 0) & (generated!=1012)]))

Original:
how far is it from denver to aspen?
Generated:
far is it from denver to aspen?
Masked word original: is
Masked word prediction: is
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Original:
what county is modesto, california in?
Generated:
what town is modesto, california in?
Masked word original: county
Masked word prediction: town
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Original:
who was galileo?
Generated:
who was he?
Masked word original: galileo
Masked word prediction: he
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Original:
what is an atom?
Generated:
what is an angel?
Masked word original: atom
Masked word prediction: angel
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Original:
when did hawaii become a state?
Generated:
when did it become a state?
Masked word original: hawaii
Masked word prediction: it
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Original:
how tall is the sears building?
Generated:
how big is the sears building?
Masked word original: tall
Masked word prediction: big
$$$

In [63]:
def calculate_wer(sequence_a: str, sequence_b: str) -> int:
    # taken from https://github.com/SeanNaren/deepspeech.pytorch/blob/master/decoder.py
    b = set(sequence_a.split() + sequence_b.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in sequence_a.split()]
    w2 = [chr(word2char[w]) for w in sequence_b.split()]

    return Lev.distance(''.join(w1), ''.join(w2))

In [26]:
class SequenceModifier:

    def __init__(
            self,
            vocab: List[str],
            remove_prob: float = 0.05,
            add_prob: float = 0.05,
            replace_prob: float = 0.1
    ) -> None:
        assert sum([remove_prob, add_prob, replace_prob]) > 0.0
        self.vocab = vocab
        self.remove_prob = remove_prob
        self.add_prob = add_prob
        self.replace_prob = replace_prob

    def remove_token(self, sequence: List[str]) -> List[str]:
        samples = np.random.binomial(n=1, p=self.remove_prob, size=len(sequence))
        sequence = [token for i, token in enumerate(sequence) if not samples[i]]
        return sequence

    def replace_token(self, sequence: List[str]) -> List[str]:
        samples = np.random.binomial(n=1, p=self.replace_prob, size=len(sequence))
        new_sequence = [random.choice(self.vocab) if samples[i] else sequence[i] for i in range(len(sequence))]
        return new_sequence

    def add_token(self, sequence: List[str]) -> List[str]:
        new_sequence = sequence + [
            random.choice(self.vocab)
            for _ in range(np.random.binomial(len(sequence), self.add_prob))
        ]
        return new_sequence

    def __call__(self, sequence: str) -> str:
        splitted_sequence = sequence.split()
        if len(splitted_sequence) > 1 and self.remove_prob:
            splitted_sequence = self.remove_token(splitted_sequence)

        if self.replace_prob:
            splitted_sequence = self.replace_token(splitted_sequence)

        if self.add_prob:
            splitted_sequence = self.add_token(splitted_sequence)
        return " ".join(splitted_sequence)

In [27]:
sentences = generated_batch_sentence + original_batch_sentence

In [28]:
dataset_dl = []
non_adversarial_indexes = np.random.randint(0, len(sentences), size=(400, 2))
for id1, id2 in tqdm(non_adversarial_indexes):
    tr1 = sentences[id1].strip()
    tr2 = sentences[id2].strip()
    
    dist = calculate_wer(tr1, tr2)
    ex = {"seq_a": tr1, "seq_b": tr2, "dist": dist}
    dataset_dl.append(ex)

In [29]:
len(dataset_dl)

400

In [30]:
vocab = []
for seq in sentences:
    vocab.extend(seq.split())

vocab = list(set(vocab))
NUM_SMALL_CHANGES = 3
NUM_SEQUENTIAL_CHANGES = 10

REMOVE_PROB = 0.0
ADD_PROB = 0.0
REPLACE_PROB = 1.0

modifier = SequenceModifier(
    vocab,
    remove_prob = REMOVE_PROB,
    add_prob = ADD_PROB,
    replace_prob = REPLACE_PROB
)
adversarial_indexes = np.random.randint(0, len(sentences), size=(100, ))
for idx in tqdm(adversarial_indexes):
    tr1 = sentences[idx].strip()

    for _ in range(NUM_SMALL_CHANGES):
        tr2 = modifier(tr1).strip()
        dist = calculate_wer(tr1, tr2)
        ex = {"seq_a": tr1, "seq_b": tr2, "dist": dist}
        dataset_dl.append(ex)

    tr2 = tr1
    num_changes = NUM_SEQUENTIAL_CHANGES
    if num_changes > len(tr1.split()):
        num_changes = len(tr1.split())
    for _ in range(num_changes):
        position_idx = random.randint(0, len(tr1.split()) - 1)
        replace_with = random.choice(vocab)
        tr2 = tr2.split()
        tr2[position_idx] = replace_with
        tr2 = " ".join(tr2)

        dist = calculate_wer(tr1, tr2)
        ex = {"seq_a": tr1, "seq_b": tr2, "dist": dist}
        dataset_dl.append(ex)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train_path = "/content/gdrive/My Drive/train.json"
test_path = "/content/gdrive/My Drive/test.json"

In [ ]:
train, test = train_test_split(dataset_dl, test_size=0.25)
with jsonlines.open(train_path, "w") as writer:
    for ex in train:
        writer.write(ex)

with jsonlines.open(test_path, "w") as writer:
    for ex in test:
        writer.write(ex)

In [31]:
len(dataset_dl)

1316

In [33]:
for i in dataset_dl:
    print(i)
    break

{'seq_a': 'what is the average speed of the horses at the kentucky derby?', 'seq_b': 'president bush had a small interest in which baseball team?', 'dist': 12}


In [43]:
import torch
from torch.utils.data import Dataset as TorchDataset

class Dataset(TorchDataset):
    
    """Custom data.Dataset compatible with data.DataLoader."""
    def __init__(self, data, index=0, batch_size=32):
        self.data = data
        self.index = index
        self.batch_size = batch_size
    def __getitem__(self, data):

        global index
        data = self.data
        batch_size = self.batch_size
        a_list = []
        b_list = []
        dist_list = []
        for _ in range(batch_size):
            
            word_a = data[self.index]['seq_a']
            word_a = torch.Tensor(tokenizer.encode(word_a, padding='max_length', truncation=True)).unsqueeze(0)
            word_b = data[self.index]['seq_b']
            word_b = torch.Tensor(tokenizer.encode(word_b, padding='max_length', truncation=True)).unsqueeze(0)
            word_dis = data[self.index]['dist']
            
            word_dis= torch.Tensor([word_dis]).unsqueeze(0)
            # print(word_dis.shape)
            a_list.append(word_a)
            b_list.append(word_b)
            dist_list.append(word_dis)
            self.index = (self.index + 1) % len(data)

        tens_a = torch.cat(a_list, dim=0)
        tens_b = torch.cat(b_list, dim=0)

        tens_dis = torch.cat(dist_list, dim=0)
        # print(tens_dis.shape)
        return tens_a, tens_b, tens_dis
        

    def __len__(self):
        return len(self.data) // self.batch_size

In [44]:
index = 0 
initial_ = Dataset(dataset_dl)

In [47]:
loader =  DataLoader(initial_, batch_size=1)
for batch in loader:
    print(batch[0].shape, batch[1].shape, batch[2].shape)

torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) torch.Size([1, 32, 1])
torch.Size([1, 32, 512]) torch.Size([1, 32, 512]) t

In [ ]:
# class Deep_lev(torch.nn.Module):

#     def __init__(self, vocab_size, embedding_dim, hidden_dim) :
#         super().__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=False)

#         self.linear = nn.Linear(hidden_dim * 3, 1)
#         self._loss = nn.MSELoss()


#     def encode_sequence(self, sequence):
#         embedded_sequence = self.embeddings(sequence)
#         out, (ht, ct) = self.encoder(embedded_sequence)
#         return ht[-1]

#     def forward(self, sequence_a, sequence_b, distance):
#         embedded_sequence_a = self.encode_sequence(sequence_a)
#         embedded_sequence_b = self.encode_sequence(sequence_b)
#         diff = torch.abs(embedded_sequence_a - embedded_sequence_b)
#         representation = torch.cat([embedded_sequence_a, embedded_sequence_b, diff], dim=-1)

#         approx_distance = self.linear(representation)
#         output_dict = {"distance": approx_distance}

#         if distance is not None:
#             output_dict["loss"] = self._loss(approx_distance.view(-1), distance.view(-1))
#         return output_dict

In [73]:
em = nn.Embedding(512, 128)

In [86]:
for batch in loader:
    print(batch[0][0][0])
    em(torch.LongTensor([batch[0][0][0]]))
    break

tensor([  101.,  2054.,  2711.,  1005.,  1055.,  2132.,  2003.,  2006.,  1037.,
        27211.,  1029.,   102.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0., 

ValueError: ignored